# LSTMs for Human Activity Recognition Time Series Classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Load the dataset

In [2]:
from utils import load_dataset

In [3]:
X_train, y_train = load_dataset(group='train', prefix='HARDataset/')
X_test, y_test = load_dataset(group='test', prefix='HARDataset/')
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 128, 9), (7352, 1), (2947, 128, 9), (2947, 1))

In [4]:
y_train = tf.keras.utils.to_categorical(y_train - 1)
y_test = tf.keras.utils.to_categorical(y_test - 1)

In [5]:
y_train.shape, y_test.shape

((7352, 6), (2947, 6))

## Fit and evaluate the model

In [6]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [7]:
def create_lstm_model():
  return tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(n_timesteps, n_features)),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(n_outputs)
  ])

In [8]:
model = create_lstm_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               44000     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [10]:
EPOCHS = 15
BATCH_SIZE = 64

In [11]:
model.fit(X_train,
          y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=(X_test, y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 4s 581us/sample - loss: 1.2221 - accuracy: 0.4825 - val_loss: 0.9725 - val_accuracy: 0.5887
Epoch 2/15
7352/7352 [==============================] - 2s 309us/sample - loss: 0.7867 - accuracy: 0.6527 - val_loss: 0.8335 - val_accuracy: 0.6434
Epoch 3/15
7352/7352 [==============================] - 2s 257us/sample - loss: 0.6324 - accuracy: 0.7286 - val_loss: 0.6434 - val_accuracy: 0.7618
Epoch 4/15
7352/7352 [==============================] - 2s 295us/sample - loss: 0.5509 - accuracy: 0.7967 - val_loss: 0.4877 - val_accuracy: 0.8419
Epoch 5/15
7352/7352 [==============================] - 2s 296us/sample - loss: 0.3898 - accuracy: 0.8619 - val_loss: 0.4707 - val_accuracy: 0.8198
Epoch 6/15
7352/7352 [==============================] - 2s 270us/sample - loss: 0.3304 - accuracy: 0.8808 - val_loss: 0.5433 - val_accuracy: 0.8045
Epoch 7/15
7352/7352 [==============================] - 2s 275us

## Develop a CNN-LSTM Network Model
One approach to implementing this model is to split each window of 128 time steps into subsequences for the CNN model to process. For example, the 128 time steps in each window can be split into four subsequences of 32 time steps.

In [12]:
# Reshape data into time steps of sub-sequences
n_steps, n_length = 4, 32
X_train = X_train.reshape(-1, n_steps, n_length, n_features)
X_test = X_test.reshape(-1, n_steps, n_length, n_features)

In [24]:
# Define the model
def create_cnn_lstm_model():
  return tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(None, n_length, n_features)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(pool_size=2)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(n_outputs)
  ])

In [25]:
model = create_cnn_lstm_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_15 (TimeDis (None, None, 30, 64)      1792      
_________________________________________________________________
time_distributed_16 (TimeDis (None, None, 28, 64)      12352     
_________________________________________________________________
time_distributed_17 (TimeDis (None, None, 28, 64)      0         
_________________________________________________________________
time_distributed_18 (TimeDis (None, None, 14, 64)      0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, None, 896)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               398800    
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)              

In [26]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [27]:
model.fit(X_train,
          y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=(X_test, y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 3s 416us/sample - loss: 0.5960 - accuracy: 0.7545 - val_loss: 0.5156 - val_accuracy: 0.8151
Epoch 2/15
7352/7352 [==============================] - 1s 173us/sample - loss: 0.2045 - accuracy: 0.9223 - val_loss: 0.4343 - val_accuracy: 0.8833
Epoch 3/15
7352/7352 [==============================] - 1s 184us/sample - loss: 0.1471 - accuracy: 0.9429 - val_loss: 0.4142 - val_accuracy: 0.8809
Epoch 4/15
7352/7352 [==============================] - 1s 175us/sample - loss: 0.1247 - accuracy: 0.9484 - val_loss: 0.4043 - val_accuracy: 0.8945
Epoch 5/15
7352/7352 [==============================] - 1s 194us/sample - loss: 0.1423 - accuracy: 0.9442 - val_loss: 0.3803 - val_accuracy: 0.8965
Epoch 6/15
7352/7352 [==============================] - 1s 177us/sample - loss: 0.1156 - accuracy: 0.9535 - val_loss: 0.4174 - val_accuracy: 0.8921
Epoch 7/15
7352/7352 [==============================] - 2s 207us

## ConvLSTM Network Model

In [28]:
X_train = X_train.reshape(-1, n_steps, 1, n_length, n_features)
X_test = X_test.reshape(-1, n_steps, 1, n_length, n_features)
X_train.shape, X_test.shape

((7352, 4, 1, 32, 9), (2947, 4, 1, 32, 9))

In [31]:
def create_conv_lstm_model():
  return tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(n_steps, 1, n_length, n_features)),
    tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1, 3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(n_outputs)
  ])

In [32]:
model = create_conv_lstm_model()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 30, 64)         56320     
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 30, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               192100    
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 606       
Total params: 249,026
Trainable params: 249,026
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [34]:
model.fit(X_train,
          y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=(X_test, y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 4s 608us/sample - loss: 0.5937 - accuracy: 0.7667 - val_loss: 0.5572 - val_accuracy: 0.8320
Epoch 2/15
7352/7352 [==============================] - 3s 365us/sample - loss: 0.2060 - accuracy: 0.9225 - val_loss: 0.4451 - val_accuracy: 0.8778
Epoch 3/15
7352/7352 [==============================] - 3s 359us/sample - loss: 0.1538 - accuracy: 0.9403 - val_loss: 0.5302 - val_accuracy: 0.8748
Epoch 4/15
7352/7352 [==============================] - 3s 365us/sample - loss: 0.1301 - accuracy: 0.9453 - val_loss: 0.5790 - val_accuracy: 0.8836
Epoch 5/15
7352/7352 [==============================] - 3s 366us/sample - loss: 0.1262 - accuracy: 0.9470 - val_loss: 0.3857 - val_accuracy: 0.8951
Epoch 6/15
7352/7352 [==============================] - 3s 351us/sample - loss: 0.1169 - accuracy: 0.9505 - val_loss: 0.4752 - val_accuracy: 0.8935
Epoch 7/15
7352/7352 [==============================] - 3s 363us